In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow import keras
from sklearn import model_selection
from sklearn.metrics import classification_report
import imgaug.augmenters as iaa
import imgaug as ia
import os

In [ ]:
#unzip miniImageNet dataset
!unzip '/content/drive/MyDrive/miniimagenet/mini/mini-imagenet.zip' -d '/content'
!rm /content/images/.DS_Store

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/images/n0679411000000879.jpg  
  inflating: /content/images/n0679411000000880.jpg  
  inflating: /content/images/n0679411000000881.jpg  
  inflating: /content/images/n0679411000000882.jpg  
  inflating: /content/images/n0679411000000885.jpg  
  inflating: /content/images/n0679411000000894.jpg  
  inflating: /content/images/n0679411000000895.jpg  
  inflating: /content/images/n0679411000000897.jpg  
  inflating: /content/images/n0679411000000898.jpg  
  inflating: /content/images/n0679411000000899.jpg  
  inflating: /content/images/n0679411000000900.jpg  
  inflating: /content/images/n0679411000000901.jpg  
  inflating: /content/images/n0679411000000902.jpg  
  inflating: /content/images/n0679411000000904.jpg  
  inflating: /content/images/n0679411000000905.jpg  
  inflating: /content/images/n0679411000000909.jpg  
  inflating: /content/images/n0679411000000912.jpg  
  inflating: /content/images/n0679411000000913.jpg  
  inflating: /conte

In [ ]:
ia.seed(1)
def Data_Aug(samples):
    randomaug = iaa.Sometimes(
        0.5,
        iaa.OneOf([
            iaa.Sequential([
                iaa.AllChannelsCLAHE(clip_limit=(1, 10)),
                iaa.Affine(rotate=(-5, 5))
            ]),
            iaa.Sequential([
                iaa.Affine(translate_percent={
                    "x": (-0.2, 0.2),
                    "y": (-0.2, 0.2)
                }),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.Affine(rotate=(-10, 10)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.PerspectiveTransform(scale=(0, 0.15)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ])
        ]))


    return randomaug.augment_images(samples)

def resize_images(imgs, hight, width):
    seq = iaa.Resize({"height": hight, "width": width})
    aug = seq.augment_images(imgs)
    return aug

In [ ]:
#load dataset
def load_data(image_dir, scv_dir):
  data = pd.read_csv(scv_dir)
  data_x = []
  data_y = []
  for name,label in zip(data['filename'],data['label']):
    img = keras.preprocessing.image.load_img(os.path.join(image_dir, name),target_size=(84,84))
    img = keras.preprocessing.image.img_to_array(img,dtype='uint8')
    data_x.append(img)
    data_y.append(label)
  data_x = np.array(data_x)
  data_y = np.array(data_y)
  return data_x, data_y
class minidata(keras.utils.Sequence):
    def __init__(self, data_x, data_y, batch_size, train):
      self.data_x = data_x
      self.data_y = data_y
      self.train = train
      self.batch_size = batch_size

    def __len__(self):
      return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):
      batch_x = self.data_x[idx * self.batch_size:(idx + 1) * self.batch_size]
                            
      batch_y = self.data_y[idx * self.batch_size:(idx + 1) * self.batch_size]
      if self.train == True:
          batch_x = Data_Aug(batch_x)

      batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
      batch_y = keras.utils.to_categorical(batch_y, 100)
      return batch_x, [batch_y, batch_y, batch_y, batch_y, batch_y]

In [ ]:
#save the best model during training 
class CustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(CustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_model_accuracy') + logs.get(
            'val_model_1_accuracy') + logs.get(
                'val_model_2_accuracy') + logs.get(
                    'val_model_3_accuracy') + logs.get('val_model_4_accuracy')

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
#Cosine&Euclidean distance loss
def descriptor(X, mask = False, max_norm = False):
  X = tf.reduce_mean(X, axis=3)
  if mask:
    mean_X = tf.reduce_mean(X, axis = [1,2], keepdims= True)
    X = tf.where(X > mean_X, X , tf.zeros_like(X, dtype=tf.float32))
  if max_norm:
    max_X=tf.reduce_max(X, axis = [1,2], keepdims= True)
    X = tf.math.divide_no_nan(X, max_X)
  return X 
  
def compute_euclidean_sum(X, Y):
  L2_distance = tf.reduce_mean(tf.exp(-(X - Y)**2), axis= [1,2])
  return tf.reduce_mean(L2_distance)

def compute_cosine_sum(X, Y):
  X = tf.reshape(X, [tf.shape(X)[0], -1])
  Y = tf.reshape(Y, [tf.shape(Y)[0], -1])
  loss = -keras.losses.cosine_similarity(X,Y,axis=-1)
  return tf.reduce_mean(loss)

def cosine_euclidean_sum_loss(x, cosine_weight, euclidean_weight, mask, max_norm):
  descriptor0 = descriptor(x[0], mask=mask, max_norm=max_norm)
  descriptor1 = descriptor(x[1], mask=mask, max_norm=max_norm)
  descriptor2 = descriptor(x[2], mask=mask, max_norm=max_norm)
  descriptor3 = descriptor(x[3], mask=mask, max_norm=max_norm)
  descriptor4 = descriptor(x[4], mask=mask, max_norm=max_norm)
  cosine_sum = compute_cosine_sum(descriptor0, descriptor1)+compute_cosine_sum(descriptor0, descriptor2)+compute_cosine_sum(descriptor0, descriptor3)+compute_cosine_sum(descriptor0, descriptor4)+compute_cosine_sum(descriptor1, descriptor2)+compute_cosine_sum(descriptor1, descriptor3)+compute_cosine_sum(descriptor1, descriptor4)+compute_cosine_sum(descriptor2, descriptor3)+compute_cosine_sum(descriptor2, descriptor4)+compute_cosine_sum(descriptor3, descriptor4)
  euclidean_sum = compute_euclidean_sum(descriptor0, descriptor1)+compute_euclidean_sum(descriptor0, descriptor2)+compute_euclidean_sum(descriptor0, descriptor3)+compute_euclidean_sum(descriptor0, descriptor4)+compute_euclidean_sum(descriptor1, descriptor2)+compute_euclidean_sum(descriptor1, descriptor3)+compute_euclidean_sum(descriptor1, descriptor4)+compute_euclidean_sum(descriptor2, descriptor3)+compute_euclidean_sum(descriptor2, descriptor4)+compute_euclidean_sum(descriptor3, descriptor4)
  return cosine_sum, euclidean_sum, cosine_weight*cosine_sum+euclidean_weight*euclidean_sum  


In [ ]:
#create single ResNet model
def creat_ResNet(input_shape, num_classes=100, num_filters = 64, random_seed=None):
  inputs = keras.Input(shape=input_shape)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x)

  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(2*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x) 

  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(4*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x) 

  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(8*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  last_acti = keras.layers.MaxPooling2D((2,2))(x) 

  x = keras.layers.GlobalAveragePooling2D()(last_acti)
  outputs = keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  model = keras.Model(inputs=inputs, outputs=[outputs,last_acti])
  return model

In [ ]:
#load miniImage images
train_dir = '/content/drive/MyDrive/miniimagenet/new_train.csv'
val_dir = '/content/drive/MyDrive/miniimagenet/new_val.csv'
test_dir = '/content/drive/MyDrive/miniimagenet/new_test.csv'
image_dir = '/content/images'
x_train, y_train = load_data(image_dir, train_dir)
x_val, y_val = load_data(image_dir, val_dir)
x_test, y_test = load_data(image_dir, test_dir)
_, x_val, _, y_val = model_selection.train_test_split(
    x_val,
    y_val,
    test_size=2000,
    random_state=0,
    stratify=y_val)
#select training samples
train_index = []
random.seed(0)
for i in range(100):
    train_index += (random.sample(list(np.where(y_train == i)[0]), 100))
random.shuffle(train_index)
sampled_x_train = x_train[train_index]
sampled_y_train = y_train[train_index]
del train_index

In [ ]:
#construct joint training of base models
input_size = (84, 84, 3)
model0 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=1)
model1 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=2)
model2 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=3)
model3 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=4)
model4 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=5)
allinputs = keras.Input(shape=input_size)
output0, feature_maps0 = model0(allinputs)
output1, feature_maps1 = model1(allinputs)
output2, feature_maps2 = model2(allinputs)
output3, feature_maps3 = model3(allinputs)
output4, feature_maps4 = model4(allinputs)
model_train = keras.Model(
    inputs=allinputs, outputs=[output0, output1, output2, output3, output4])

#implement distance loss
loss1, loss2, loss = cosine_euclidean_sum_loss(x=[feature_maps0, feature_maps1, feature_maps2, feature_maps3, feature_maps4], 
                                      cosine_weight=1, euclidean_weight=10, mask=True, max_norm=False)

model_train.add_loss(loss)
model_train.add_metric(loss1, name="loss1", aggregation='mean')
model_train.add_metric(loss2, name="loss2", aggregation='mean')

model_train.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],
                    metrics=['accuracy'])


#construct training and validation samples
train_sequence = minidata(sampled_x_train, sampled_y_train, batch_size=10, train=True)
validation_sequence = minidata(x_val, y_val, batch_size=10, train=False)

#training
history = model_train.fit(
    train_sequence,
    epochs=200,
    validation_data=validation_sequence,
    callbacks=CustomEarlyStoppingAndSave(
        save_path='100_samples_seed_fix.h5',
        patience=10))
frame = pd.DataFrame({
    'loss':
    history.history['loss'],
    'loss1':
    history.history['loss1'],
    'loss2':
    history.history['loss2'],
    'model_0_loss':
    history.history['model_loss'],
    'model_1_loss':
    history.history['model_1_loss'],
    'model_2_loss':
    history.history['model_2_loss'],
    'model_3_loss':
    history.history['model_3_loss'],
    'model_4_loss':
    history.history['model_4_loss'],
    'model_0_accuracy':
    history.history['model_accuracy'],
    'model_1_accuracy':
    history.history['model_1_accuracy'],
    'model_2_accuracy':
    history.history['model_2_accuracy'],
    'model_3_accuracy':
    history.history['model_3_accuracy'],
    'model_4_accuracy':
    history.history['model_4_accuracy'],
    'val_loss':
    history.history['val_loss'],
    'val_loss1':
    history.history['val_loss1'],
    'val_loss2':
    history.history['val_loss2'],
    'val_model_0_loss':
    history.history['val_model_loss'],
    'val_model_1_loss':
    history.history['val_model_1_loss'],
    'val_model_2_loss':
    history.history['val_model_2_loss'],
    'val_model_3_loss':
    history.history['val_model_3_loss'],
    'val_model_4_loss':
    history.history['val_model_4_loss'],
    'val_model_0_accuracy':
    history.history['val_model_accuracy'],
    'val_model_1_accuracy':
    history.history['val_model_1_accuracy'],
    'val_model_2_accuracy':
    history.history['val_model_2_accuracy'],
    'val_model_3_accuracy':
    history.history['val_model_3_accuracy'],
    'val_model_4_accuracy':
    history.history['val_model_4_accuracy'],
})
#frame.to_excel('/content/drive/MyDrive/miniinmagenetexperiment/ResNet/5models200s100samples1e-3bs10/table_100_samples_seed_fix(2).xlsx')


Epoch 1/200
30/30 [==============================] - 31s 413ms/step - loss: 32.8006 - model_loss: 6.7744 - model_1_loss: 6.5074 - model_2_loss: 6.5576 - model_3_loss: 6.3853 - model_4_loss: 6.5758 - model_accuracy: 0.0033 - model_1_accuracy: 0.0133 - model_2_accuracy: 0.0000e+00 - model_3_accuracy: 0.0067 - model_4_accuracy: 0.0033 - loss1: 7.7196 - loss2: 8.3221 - val_loss: 9948.7910 - val_model_loss: 1709.5825 - val_model_1_loss: 3080.3269 - val_model_2_loss: 2062.6497 - val_model_3_loss: 515.1663 - val_model_4_loss: 2581.0613 - val_model_accuracy: 0.0095 - val_model_1_accuracy: 0.0100 - val_model_2_accuracy: 0.0100 - val_model_3_accuracy: 0.0100 - val_model_4_accuracy: 0.0100 - val_loss1: 8.9410 - val_loss2: 4.3494
Epoch 2/200
30/30 [==============================] - 10s 338ms/step - loss: 22.7857 - model_loss: 4.6826 - model_1_loss: 4.5427 - model_2_loss: 4.5169 - model_3_loss: 4.4946 - model_4_loss: 4.5488 - model_accuracy: 0.0167 - model_1_accuracy: 0.0233 - model_2_accuracy: 0.0

In [ ]:
#evaluation of base models
x_test = x_test/255.0
evaluate = model_train.evaluate(x_test,[keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test)],batch_size=10)
evaluate

1200/1200 [==============================] - 43s 34ms/step - loss: 45.3372 - model_loss: 8.9183 - model_1_loss: 8.6927 - model_2_loss: 9.0622 - model_3_loss: 9.3440 - model_4_loss: 9.3199 - model_accuracy: 0.0661 - model_1_accuracy: 0.0648 - model_2_accuracy: 0.0621 - model_3_accuracy: 0.0704 - model_4_accuracy: 0.0577 - loss1: 8.6154 - loss2: 8.6079


In [ ]:
#hard voting ensemble
class HardVotingEnsemble:
    def __init__(self, model, testdata ,batch_size):
        self.ensemble_model = model
        self.testdata = testdata
        self.batch_size = batch_size
        self.data_size = testdata[1].shape[0]
    def prediction(self):
        predictions =[]
        for i in range(int(self.data_size/self.batch_size)):
            batch_predictions = np.argmax(self.ensemble_model.predict(self.testdata[0][i*self.batch_size:(i+1)*self.batch_size]),axis=2) 
            batch_predictions = np.stack(batch_predictions, axis=1)
            batch_predictions = [np.argmax(np.bincount(pre)) for pre in batch_predictions]
            predictions = predictions+batch_predictions
        return np.array(predictions)

    def evaluate(self):
        predictions = self.prediction()
        accuracy = np.sum(
            predictions == self.testdata[1]) / self.data_size
        return accuracy
HEnsemble = HardVotingEnsemble(model = model_train,testdata = (x_test, y_test),batch_size=10)
HEnsemble.evaluate()

0.07575

In [ ]:
#soft voting ensemble
def SoftVotingEnsemble(model, x, y):
  data_size = x.shape[0]
  predictions = tf.argmax(tf.reduce_mean(model.predict(x), axis = 0),axis=1)
  accuracy = np.sum(
     predictions == y) / data_size
  return accuracy
SoftVotingEnsemble(model_train, x_test, y_test)

0.07808333333333334

In [ ]:
class eCustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(eCustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        #self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_accuracy') 

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            #self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
class eminidata(keras.utils.Sequence):
    def __init__(self, data_x, data_y, batch_size, train):
      self.data_x = data_x
      self.data_y = data_y
      self.train = train
      self.batch_size = batch_size

    def __len__(self):
      return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):
      batch_x = self.data_x[idx * self.batch_size:(idx + 1) * self.batch_size]
                            
      batch_y = self.data_y[idx * self.batch_size:(idx + 1) * self.batch_size]
      if self.train == True:
          batch_x = Data_Aug(batch_x)

      batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
      batch_y = keras.utils.to_categorical(batch_y, 100)
      return batch_x, batch_y

In [ ]:
#feature fusion model
model_path = '100_samples_seed_fix.h5'
model = keras.models.load_model(model_path)
extractor_1 = keras.models.Model(inputs=model.layers[1].inputs,outputs=model.layers[1].layers[56].output)
extractor_2 = keras.models.Model(inputs=model.layers[2].inputs,outputs=model.layers[2].layers[56].output)
extractor_3 = keras.models.Model(inputs=model.layers[3].inputs,outputs=model.layers[3].layers[56].output)
extractor_4 = keras.models.Model(inputs=model.layers[4].inputs,outputs=model.layers[4].layers[56].output)
extractor_5 = keras.models.Model(inputs=model.layers[5].inputs,outputs=model.layers[5].layers[56].output)
extractor_1.trainable = False
extractor_2.trainable = False
extractor_3.trainable = False
extractor_4.trainable = False
extractor_5.trainable = False
inputs = keras.Input(shape = input_size)
features_1 = extractor_1(inputs, training = False)
features_2 = extractor_2(inputs, training = False)
features_3 = extractor_3(inputs, training = False)
features_4 = extractor_4(inputs, training = False)
features_5 = extractor_5(inputs, training = False)

x = keras.layers.Concatenate(axis=-1)([features_1,features_2,features_3,features_4,features_5])

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(512,  name='dense_100')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(100, activation='softmax', name='dense_101')(x)
ensemble = keras.Model(inputs = inputs, outputs = outputs)
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
train_sequence = eminidata(sampled_x_train, sampled_y_train, batch_size=10, train=True)
validation_sequence = eminidata(x_val, y_val, batch_size=10, train=False)
history = ensemble.fit(
    train_sequence,
    epochs=50,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/50
30/30 [==============================] - 14s 316ms/step - loss: 5.1245 - accuracy: 0.0133 - val_loss: 6.8937 - val_accuracy: 0.0400
Epoch 2/50
30/30 [==============================] - 8s 268ms/step - loss: 3.3191 - accuracy: 0.2967 - val_loss: 5.3195 - val_accuracy: 0.0540
Epoch 3/50
30/30 [==============================] - 8s 263ms/step - loss: 2.4934 - accuracy: 0.4600 - val_loss: 4.9232 - val_accuracy: 0.0575
Epoch 4/50
30/30 [==============================] - 8s 264ms/step - loss: 2.0947 - accuracy: 0.5900 - val_loss: 4.7829 - val_accuracy: 0.0670
Epoch 5/50
30/30 [==============================] - 7s 249ms/step - loss: 1.6412 - accuracy: 0.7333 - val_loss: 4.7852 - val_accuracy: 0.0640
Epoch 6/50
30/30 [==============================] - 8s 265ms/step - loss: 1.3076 - accuracy: 0.8133 - val_loss: 4.6440 - val_accuracy: 0.0745
Epoch 7/50
30/30 [==============================] - 7s 256ms/step - loss: 1.1196 - accuracy: 0.8500 - val_loss: 4.6328 - val_accuracy: 0.0705
Epoch

In [ ]:
#finetuning
extractor_1.trainable = True
extractor_2.trainable = True
extractor_3.trainable = True
extractor_4.trainable = True
extractor_5.trainable = True
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
#train_sequence = ecifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=False)
#validation_sequence = ecifar10data(x_val, y_val, batch_size=10, train=False)
history = ensemble.fit(
    train_sequence,
    epochs=20,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/20
30/30 [==============================] - 19s 372ms/step - loss: 0.2937 - accuracy: 0.9733 - val_loss: 4.6471 - val_accuracy: 0.0815
Epoch 2/20
30/30 [==============================] - 9s 309ms/step - loss: 0.2914 - accuracy: 0.9733 - val_loss: 4.6455 - val_accuracy: 0.0785
Epoch 3/20
30/30 [==============================] - 9s 309ms/step - loss: 0.2292 - accuracy: 0.9667 - val_loss: 4.6482 - val_accuracy: 0.0775
Epoch 4/20
30/30 [==============================] - 9s 307ms/step - loss: 0.1729 - accuracy: 0.9833 - val_loss: 4.6520 - val_accuracy: 0.0790
Epoch 5/20
30/30 [==============================] - 9s 307ms/step - loss: 0.1683 - accuracy: 0.9967 - val_loss: 4.6575 - val_accuracy: 0.0795
Epoch 6/20
30/30 [==============================] - 9s 307ms/step - loss: 0.2702 - accuracy: 0.9733 - val_loss: 4.6755 - val_accuracy: 0.0785
Epoch 7/20
30/30 [==============================] - 9s 312ms/step - loss: 0.1917 - accuracy: 0.9733 - val_loss: 4.6806 - val_accuracy: 0.0790
Epoch

In [ ]:
#f1 score
predict = tf.argmax(ensemble.predict(x_test),axis=-1)
classification_report(y_test, predict,digits=4)

'              precision    recall  f1-score   support\n\n           0     0.0323    0.0167    0.0220       120\n           1     0.0870    0.1500    0.1101       120\n           2     0.0515    0.0417    0.0461       120\n           3     0.2330    0.2000    0.2152       120\n           4     0.0556    0.0250    0.0345       120\n           5     0.2500    0.2833    0.2656       120\n           6     0.0167    0.0167    0.0167       120\n           7     0.3092    0.3917    0.3456       120\n           8     0.0652    0.0500    0.0566       120\n           9     0.0594    0.1083    0.0767       120\n          10     0.1700    0.2833    0.2125       120\n          11     0.0358    0.0917    0.0515       120\n          12     0.0440    0.0333    0.0379       120\n          13     0.0455    0.0167    0.0244       120\n          14     0.0535    0.0833    0.0651       120\n          15     0.1020    0.0417    0.0592       120\n          16     0.0842    0.1333    0.1032       120\n       